In [5]:
%load_ext autoreload
%autoreload
%run -i redistricter.py

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
random seed 1000
MCMC 1 ... pushing ... pop_imbalance=88.18 ... setting tol=88.19% ... recombed cd 10 & 08 got pop_imbalance=84.42% ... success
MCMC 2 ... pushing ... pop_imbalance=84.42 ... setting tol=84.43% ... recombed cd 25 & 31 got pop_imbalance=81.82% ... success
MCMC 3 ... pushing ... pop_imbalance=81.82 ... setting tol=81.83% ... recombed cd 25 & 21 got pop_imbalance=78.29% ... success
MCMC 4 ... pushing ... pop_imbalance=78.29 ... setting tol=78.30% ... recombed cd 08 & 10 got pop_imbalance=78.19% ... success
MCMC 5 ... pushing ... pop_imbalance=78.19 ... setting tol=78.20% ... recombed cd 08 & 10 got pop_imbalance=69.78% ... success
MCMC 6 ... pushing ... pop_imbalance=69.78 ... setting tol=69.79% ... recombed cd 08 & 35 got pop_imbalance=65.47% ... success
MCMC 7 ... pushing ... pop_imbalance=65.47 ... setting tol=65.48% ... recombed cd 23 & 08 got pop_imbalance=64.80% ... success
MCMC 8

KeyboardInterrupt: 

In [3]:
from . import *

try:
    import pandas_bokeh
except:
    os.system('pip install --upgrade pandas-bokeh')
    import pandas_bokeh

@dataclasses.dataclass
class Analysis(Base):
    nodes : str
    plans : str
        
    def plot(self):
        df = read_table(tbl=self.plans)
#         d = len(str(df['plan'].max()))
#         df['plan'] = df['plan'].astype(str).str.rjust(d, '0')
        df = df.pivot(index='geoid', columns='plan').astype(int)
        df.columns = df.columns.droplevel().rename(None)
        plans = df.columns.tolist()
        shapes = run_query(f'select geoid, total_pop, aland, polygon from {self.nodes}')
        df = df.merge(shapes, on='geoid')
        geo = gpd.GeoSeries.from_wkt(df['polygon'], crs='EPSG:4326').simplify(0.001).buffer(0) #<-- little white space @ .001 ~5.7 mb, minimal at .0001 ~10mb, with no white space ~37mb
        # geo = gpd.GeoSeries.from_wkt(df['polygon'], crs='EPSG:4326').buffer(0) # <-------------------- to not simplify at all
        self.gdf = gpd.GeoDataFrame(df.drop(columns='polygon'), geometry=geo)

        pandas_bokeh.output_notebook() #<------------- uncommment to view in notebook
        
        fig = self.gdf.plot_bokeh(
            figsize=(900, 600),
            slider = plans,
            slider_name = "PLAN #",
            show_colorbar=False,
            colormap= "Category20",
            hovertool_columns=['total_pop', 'aland'],
            tile_provider="CARTODBPOSITRON",
            return_html=False,
            show_figure=True,
            **{'fill_alpha' :.8,
              'line_alpha':.05,}
        )
        return fig


    def get_results(self):
        cols = [c for c in get_cols(self.nodes) if c not in Levels + District_types + ['polygon', 'aland', 'perim', 'polsby-popper', 'point']]
        query = f"""
select
    A.cd,
    A.plan,
    {join_str(1).join([f'sum(B.{c}) as {c}' for c in cols])}
from
    {self.plans} as A
left join
    {self.nodes} as B
on
    A.geoid = B.geoid
group by
    cd, plan
"""
        self.results = run_query(query)
        
        
nodes = 'cmat-315920.redistricting_data.nodes_TX_2020_cntyvtd_cd'
plans = 
        

user_name (default=cook) 
random_seed (default=1) 


cmat-315920.redistricting_data.cook_TX_2020_cntyvtd_cd_2021-08-30_12:37:52.465105


AssertionError: 